# Get all sets

In [1]:
import itertools
from collections import namedtuple

In [2]:
# card namedtuple type
Card = namedtuple('card', ['number', 'color', 'shape', 'shade'])

In [3]:
# characteristics
numbers = ['one', 'two', 'three']
colors = ['green', 'purple', 'red']
shapes = ['diamond', 'squiggle', 'oval']
shades = ['open', 'solid', 'striped']

In [4]:
# create all cards 
all_cards = {
    Card(number=number, color=color, shape=shape, shade=shade)
    for number in numbers
    for color in colors
    for shape in shapes
    for shade in shades
}

In [5]:
print(f"There are {len(all_cards)} cards in total.")

There are 81 cards in total.


In [6]:
all_cards_list = list(all_cards)
possible_sets = list(itertools.combinations(all_cards_list, 3))

In [7]:
print(f"There are {len(possible_sets):,} different 3 card combinations in total.")

There are 85,320 different 3 card combinations in total.


In [8]:
def check_if_valid(candidate_set):
    set_numbers = set()
    set_colors = set()
    set_shapes = set()
    set_shades = set()
    
    for card in candidate_set:
        set_numbers.add(card.number)
        set_colors.add(card.color)
        set_shapes.add(card.shape)
        set_shades.add(card.shade)

    set_totals = {len(set_numbers), len(set_colors), len(set_shapes), len(set_shades)}
    
    if set_totals in [{1},{3},{1,3}]:
        return True
    
    return False

In [9]:
valid_sets, invalid_sets = set(), set()

for candidate_set in possible_sets:
    (valid_sets if check_if_valid(candidate_set) else invalid_sets).add(candidate_set)

In [10]:
assert len(valid_sets)+len(invalid_sets) == len(possible_sets)

In [11]:
print(f"There are {len(valid_sets):,} valid sets and {len(invalid_sets):,} invalid sets.")

There are 1,080 valid sets and 84,240 invalid sets.


# Get training dataset for DL task 
Dataset obtained from Kaggle: https://www.kaggle.com/datasets/kwisatzhaderach/set-cards

In [12]:
import os

In [13]:
def print_folder_tree(base_dir, indent=''):
    items = os.listdir(base_dir)
    
    items = [item for item in items if os.path.isdir(os.path.join(base_dir, item))]
    
    for index, item in enumerate(items):
        item_path = os.path.join(base_dir, item)
        is_last = index == len(items) - 1
        
        if is_last:
            print(indent + '└── ' + item)
            new_indent = indent + '    '
        else:
            print(indent + '├── ' + item)
            new_indent = indent + '│   '
        
        print_folder_tree(item_path, new_indent)

In [14]:
base_dir = 'dataset'
print_folder_tree(base_dir)

├── one
│   ├── green
│   │   ├── diamond
│   │   │   ├── solid
│   │   │   ├── striped
│   │   │   └── open
│   │   ├── squiggle
│   │   │   ├── solid
│   │   │   ├── striped
│   │   │   └── open
│   │   └── oval
│   │       ├── solid
│   │       ├── striped
│   │       └── open
│   ├── red
│   │   ├── diamond
│   │   │   ├── solid
│   │   │   ├── striped
│   │   │   └── open
│   │   ├── squiggle
│   │   │   ├── solid
│   │   │   ├── striped
│   │   │   └── open
│   │   └── oval
│   │       ├── solid
│   │       ├── striped
│   │       └── open
│   └── purple
│       ├── diamond
│       │   ├── solid
│       │   ├── striped
│       │   └── open
│       ├── squiggle
│       │   ├── solid
│       │   ├── striped
│       │   └── open
│       └── oval
│           ├── solid
│           ├── striped
│           └── open
├── zthree
│   ├── green
│   │   ├── diamond
│   │   │   ├── solid
│   │   │   ├── striped
│   │   │   └── open
│   │   ├── squiggle
│   │   │   ├── solid
│   │   │   ├── str

# Create a Custom Dataset Class

In [15]:
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from PIL import Image

import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

from torchvision.transforms import functional as F

In [25]:
class CardDetectionDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file) as f:
            self.annotations = json.load(f)
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        img_path = os.path.join(self.root_dir, annotation["image"])
        image = Image.open(img_path).convert("RGB")
        boxes = torch.tensor(annotation["boxes"], dtype=torch.float32)
        labels = torch.tensor(annotation["labels"], dtype=torch.int64)
        
        if self.transform:
            image = self.transform(image)
        
        target = {"boxes": boxes, "labels": labels}
        
        return image, target

In [17]:
transform = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = CardDataset(root_dir='dataset', transform=transform)

In [18]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [19]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # Change the output layer to match the number of classes

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

model = model.to(device)

/Users/vivianamarquez/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vivianamarquez/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: mps


In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

Epoch 1/25, Loss: 0.1809, Validation Accuracy: 1.0000
Epoch 2/25, Loss: 0.0085, Validation Accuracy: 1.0000
Epoch 3/25, Loss: 0.0067, Validation Accuracy: 1.0000
Epoch 4/25, Loss: 0.0056, Validation Accuracy: 1.0000
Epoch 5/25, Loss: 0.0109, Validation Accuracy: 1.0000
Epoch 6/25, Loss: 0.0083, Validation Accuracy: 0.9974
Epoch 7/25, Loss: 0.0001, Validation Accuracy: 1.0000
Epoch 8/25, Loss: 0.0002, Validation Accuracy: 1.0000
Epoch 9/25, Loss: 0.0197, Validation Accuracy: 1.0000
Epoch 10/25, Loss: 0.0003, Validation Accuracy: 1.0000
Epoch 11/25, Loss: 0.0003, Validation Accuracy: 1.0000
Epoch 12/25, Loss: 0.0001, Validation Accuracy: 1.0000
Epoch 13/25, Loss: 0.0000, Validation Accuracy: 1.0000
Epoch 14/25, Loss: 0.0001, Validation Accuracy: 1.0000
Epoch 15/25, Loss: 0.0000, Validation Accuracy: 1.0000
Epoch 16/25, Loss: 0.0000, Validation Accuracy: 1.0000
Epoch 17/25, Loss: 0.0000, Validation Accuracy: 1.0000
Epoch 18/25, Loss: 0.0000, Validation Accuracy: 1.0000
Epoch 19/25, Loss: 

In [22]:
torch.save(model.state_dict(), 'card_detection_model.pth')

In [23]:
def detect_cards(image_path, model, transform, device):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return dataset.classes[predicted.item()]

In [24]:
model.load_state_dict(torch.load('card_detection_model.pth'))
image_path = 'test_images/test_2.png'
detected_card = detect_cards(image_path, model, transform, device)
print(f'Detected Card: {detected_card}')

/var/folders/xm/mdklpdgs63zdhm8zvfg_l6g00000gn/T/ipykernel_61495/2380943978.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('card_detect

Detected Card: two
